## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 899.1 kB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.15.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-10-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 31s]
val_accuracy: 0.871833324432373

Best val_accuracy So Far: 0.9143333435058594
Total elapsed time: 00h 06m 52s


In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        20784     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 128)               3539072   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 3561626 (13.59 MB)
Trainable params: 3561626 (13.59 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.1112 - accuracy: 0.9584 - val_loss: 0.2985 - val_accuracy: 0.9135
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0707 - accuracy: 0.9749 - val_loss: 0.3257 - val_accuracy: 0.9095
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0484 - accuracy: 0.9828 - val_loss: 0.3689 - val_accuracy: 0.9077
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0329 - accuracy: 0.9884 - val_loss: 0.4840 - val_accuracy: 0.9077
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0239 - accuracy: 0.9917 - val_loss: 0.5671 - val_accuracy: 0.9080
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0222 - accuracy: 0.9920 - val_loss: 0.5183 - val_accuracy: 0.9110
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0180 - accuracy: 0.9939 - val_loss: 0.6552 - val_accurac